## Tensorflow Tutorial
Tensorflow is an open source deep learning library by Google.It is one of the most popular library out there.
This tutorial shows a simple neural network applied to one of the most famous dataset of deep learning.MNIST data set will be used.It contains handwritten digits from 0-9.Tensorflow tutorials have mnist dataset preloaded. 
Tensorflow has an amazing visualization tool known as tensorboard.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True) #one hot true does one hot encoding
tf.reset_default_graph()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Epoch is one round of forward propogation and one round of backpropogation.There are total 10 labels(0-9) and there are a total of 10,000 images.

In [2]:
epoch=10
display_epoch = 1
logs_path = 'mnist'
n_classes = 10  #Total 10 digits
batch_size = 100

Input data are taken as placeholders.These palceholders are feeded with dictionary.For x placeholder it expects input of [somebatchsize,784(28*28)] and y expects as [somebatchsize,10].The reason of giving none is that we can change our batch size.

In [3]:
with tf.name_scope('inputs') as inputs:
        x = tf.placeholder(tf.float32, shape=(None,784), name='x')
        y = tf.placeholder(tf.float32, shape=(None,10), name='y')

To initialize weights we use xavier initializer.Xavier initializer chooses values in random from a gaussian distribution based on the input and output layer.

In [4]:
def weights_xavi(shape, xavier_params = (None, None)):
    with tf.variable_scope('Weights'):
        (fan_in, fan_out) = xavier_params
        low = -4*np.sqrt(6.0/(fan_in + fan_out)) # {sigmoid:4, tanh:1} 
        high = 4*np.sqrt(6.0/(fan_in + fan_out))
        return tf.Variable(tf.random_uniform(shape, minval=low, maxval=high, dtype=tf.float32))

Neural net is a function that takes two inputs as parameters.The first parameter la expects the input layer/previous layer.The second parameter takes dimension of the previous layer and the dimension of the current layer.The function does a matrix multiplication on previous layer with the current layer weights w and then sums with bias b. 

In [5]:
def neuralnet(la,shape=[None,None]):
    with tf.name_scope('neuralnet'):
        w=weights_xavi(shape,xavier_params=(shape[0],shape[1]))
        b=weights_xavi([shape[1],],xavier_params=(0,shape[1]))
        act=tf.add(tf.matmul(la,w),b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

The network function takes in input as data.That data is feeded into the first neuralnet function with 500 nodes as the first hidden layer.Layer 2 and Layer 3 also has 500 nodes.Relu activation is used after each neuralnet function.Relu activation makes sure which neurons to fire and which shouldn't.The third layer is connected with 10 nodes which is the outplut layer for our digits(0-9).

In [6]:
def network(data):
    l1=neuralnet(data,shape=[784,500])
    l1=tf.nn.relu(l1)                          #Layer1
    l2=neuralnet(l1,shape=[500,500])
    l2=tf.nn.relu(l2)                          #Layer2
    l3=neuralnet(l2,shape=[500,500])
    l3=tf.nn.relu(l3)                          #Layer3
    l4=neuralnet(l3,shape=[500,10])
    return l4    

The loss function used is softmax_cross_entropy and the optimization function used is Adam.Adam optimizer works better than Gradient Descent algorithm.In tensorflow a graph is built and executed using tf.Session(). To execute each function and variable sess.run() is used.

In [7]:
with tf.name_scope('Model'): 
    pred = network(x)                      #we get a output of our network
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    tf.summary.scalar("Loss",cost)         #we find the cost using soft cross entopy.
with tf.name_scope('ADAM'):
    optimizer = tf.train.AdamOptimizer().minimize(cost)  #We use Adam Optimizer
with tf.name_scope('Accuracy'):
    correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))   #Calculate Accuracy
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    tf.summary.scalar('accuracy', accuracy)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()        
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())       #Initializes All the variables
    summary_writer = tf.summary.FileWriter(logs_path)  #WritesTheGraphForTensorboard
    summary_writer.add_graph(graph=sess.graph)        
    gstep=1
    for i in range(epoch):
            epoch_loss = 0             #calculate loss
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size) #Selects input and label
                #The optimizer,Cost,Tensorboard Function are Run.
                _, c,summary = sess.run([optimizer, cost,merged_summary_op], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
                summary_writer.add_summary(summary,gstep) #Writes Summary
                gstep+=1

            print('Epoch', i, 'completed out of',epoch,'loss:',epoch_loss)
    print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

Epoch 0 completed out of 10 loss: 1072.03007977
Epoch 1 completed out of 10 loss: 170.869018588
Epoch 2 completed out of 10 loss: 90.434417318
Epoch 3 completed out of 10 loss: 57.4140657142
Epoch 4 completed out of 10 loss: 48.98067365
Epoch 5 completed out of 10 loss: 49.470284244
Epoch 6 completed out of 10 loss: 42.9887020546
Epoch 7 completed out of 10 loss: 51.9657474167
Epoch 8 completed out of 10 loss: 40.9384374551
Epoch 9 completed out of 10 loss: 33.6574335661
Accuracy: 0.9674


### Make Sure to run the terminal where 'mnist' is saved.    
tensorboard --logdir='mnist'

In [8]:
print("Run the command line:\n" \
          "--> tensorboard --logdir='mnist' " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

Run the command line:
--> tensorboard --logdir='mnist' 
Then open http://0.0.0.0:6006/ into your web browser


# Tensorboard Graph Visualization
<img src="images/ann.png">

## Accuracy And Loss
<img src="images/annac.png">
We can see that accuracy increases and the loss decreases.Thus our network is working properly.